In [ ]:
import numpy as np
from os import urandom
from  keras. callbacks  import  ModelCheckpoint, LearningRateScheduler
from  keras. models  import  Model
from  keras. layers  import  Dense, Conv1D, Conv2D,Input, ReLU,Reshape, Permute, Add, Flatten, BatchNormalization, Activation, Dropout,DepthwiseConv2D
from keras.regularizers import l2
import  matplotlib. pyplot  as  plt
import time
from  keras  import  layers
import tensorflow as tf
import copy
bs = 2000

In [ ]:
Sbox = np.array([0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2])

raw_P = [0,  16, 32, 48, 1,  17, 33, 49, 2,  18, 34, 50, 3,  19, 35, 51,
     4,  20, 36, 52, 5,  21, 37, 53, 6,  22, 38, 54, 7,  23, 39, 55,
     8,  24, 40, 56, 9,  25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59,
     12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]
raw_P = np.array(raw_P)

# Big-Edian
index = np.array([63 - i for i in range(64)])
raw_P = 63 - raw_P[index]

P = np.array([np.where(raw_P == i) for i in range(64)])
P = np.squeeze(P)

# for decryption, to be test
Sbox_inverse = np.array([0x5, 0xe, 0xf, 0x8, 0xc, 0x1, 0x2, 0xd, 0xb, 0x4, 0x6, 0x3, 0x0, 0x7, 0x9, 0xa])
P_inverse = raw_P

# for updating keys
KP = np.array([(i+61) % 80 for i in range(80)])


# x shape: (-1, 4)
def get_Sbox_output_enc(x):
    n, m = np.shape(x)
    assert m == 4
    x_val = x[:, 0] * 8 + x[:, 1] * 4 + x[:, 2] * 2 + x[:, 3]
    y_val = Sbox[x_val]
    output = np.zeros((n, 4), dtype=np.uint8)
    for i in range(4):
        output[:, i] = (y_val >> (3 - i)) & 1
    # print('y_val shape is ', np.shape(y_val))
    return output


# x shape: (-1, 4)
def get_Sbox_output_dec(x):
    n, m = np.shape(x)
    assert m == 4
    x_val = x[:, 0] * 8 + x[:, 1] * 4 + x[:, 2] * 2 + x[:, 3]
    y_val = Sbox_inverse[x_val]
    output = np.zeros((n, 4), dtype=np.uint8)
    for i in range(4):
        output[:, i] = (y_val >> (3 - i)) & 1
    # print('y_val shape is ', np.shape(y_val))
    return output


# keys shape: (-1, 80)
def update_master_key(keys, round_counter):
    tp = keys[:, KP]
    new_keys = copy.deepcopy(tp)
    new_keys[:, :4] = get_Sbox_output_enc(tp[:, :4])
    round_counter_arr = np.array([(round_counter >> (4-i)) & 1 for i in range(5) ], dtype=np.uint8)
    new_keys[:, 60:65] = tp[:, 60:65] ^ round_counter_arr
    return new_keys


# keys shape: (-1, 80)
def expand_key(keys, nr):
    n, m = np.shape(keys)
    assert m == 80
    ks = np.zeros((nr+1, n, 64), dtype=np.uint8)
    ks[0] = keys[:, :64]
    for i in range(1, nr+1):
        keys = update_master_key(keys, i)
        ks[i] = keys[:, :64]
    return ks


# x shape: (-1, 64)
def sBoxLayer_enc(x):
    n, m = np.shape(x)
    assert m == 64
    output = np.zeros((n, 64), dtype=np.uint8)
    for i in range(16):
        st = 4 * i
        output[:, st:st+4] = get_Sbox_output_enc(x[:, st:st+4])
    return output


# x shape: (-1, 64)
def sBoxLayer_dec(x):
    n, m = np.shape(x)
    assert m == 64
    output = np.zeros((n, 64), dtype=np.uint8)
    for i in range(16):
        st = 4 * i
        output[:, st:st+4] = get_Sbox_output_dec(x[:, st:st+4])
    return output


# x shape: (-1, 64)
def pLayer_enc(x):
    output = x[:, P]
    return output


# x shape: (-1, 64)
def pLayer_dec(x):
    output = x[:, P_inverse]
    return output


# x shape: (-1, 64)
# subkeys shape: (-1, 64)
def enc_one_round(x, subkeys):
    y = sBoxLayer_enc(x)
    z = pLayer_enc(y)
    output = z ^ subkeys
    return output
def dec_one_round(x, subkeys):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    output = z ^ subkeys
    return output


def encrypt(x, ks):
    nr = ks.shape[0]
    y = x ^ ks[0]
    for i in range(1, nr):
        y = enc_one_round(y, ks[i])
    return y
def decrypt(x, ks):
    nr = ks.shape[0]
    y = x ^ ks[nr-1]
    for i in range(1, nr):
        y = dec_one_round(y, ks[nr - 1 - i])
    return y
def decrypt_1(x):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    return y,z
def decrypt_2(x):
    y = pLayer_dec(x)
#     z = sBoxLayer_dec(y)
    return y


def make_train_data(subkeyy,n=10**7, nr=9):
    x0 = np.frombuffer(urandom(n * 8), dtype=np.uint64)  # .reshape(-1, 1)
    p0 = np.zeros((n, 64), dtype=np.uint8)
    for i in range(64):
        off = 63 - i
        p0[:, i] = (x0 >> off) & 1
    arr = [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0],
           [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1],
           [1, 1, 1, 0], [1, 1, 1, 1]]
#     master_keys = np.frombuffer(urandom(n * 80), dtype=np.uint8).reshape(-1, 80) & 1
    subkeys = expand_key(subkeyy, nr)
    c0 = encrypt(p0, subkeys)

 

    #生成正样本
    pp_1=p0.copy()
    for j in range(4):
        pp_1[:,j+60]=pp_1[:,j+60]^arr[0][j]
    c0_1 = encrypt(pp_1, subkeys)
    c0_1_xor=np.bitwise_xor(c0,c0_1)

    c0_0_2,c0_0=decrypt_1(np.bitwise_xor(c0,c0))
    c0_1_xor_2,c0_1_xor_tran=decrypt_1(c0_1_xor)
    creal=np.concatenate((c0_0,c0_1_xor_tran),axis=1)
    creal_1=np.concatenate((c0_0_2,c0_1_xor_2),axis=1)
#     creal=np.concatenate((c0_0,np.bitwise_xor(c0_0,c0_1_xor)),axis=1) #解密一轮在异或
#     creal_1=np.concatenate((c0_1,np.bitwise_xor(c0,c0_1)),axis=1)
    # creal_0=np.concatenate((c0,c0_1),axis=1) #解密一轮直接做
    for i in range(1,14,2):
        pp=p0.copy()
        pp_1=p0.copy()
        for j in range(4):
            pp[:,j+60]=pp[:,j+60]^arr[i][j]
            pp_1[:,j+60]=pp_1[:,j+60]^arr[i+1][j]
        c1=encrypt(pp, subkeys)
        c1_1=encrypt(pp_1, subkeys)

        c1_xor=np.bitwise_xor(c0,c1)
        c1_1_xor=np.bitwise_xor(c0,c1_1)
        c1_xor_tran_2,c1_xor_tran=decrypt_1(c1_xor)
        c1_1_xor_tran_2,c1_1_xor_tran=decrypt_1(c1_1_xor)
        
        creal=np.concatenate((creal,c1_xor_tran,c1_1_xor_tran),axis=1)
        creal_1=np.concatenate((creal_1,c1_xor_tran_2,c1_1_xor_tran_2),axis=1)

#         creal=np.concatenate((creal,np.bitwise_xor(c0_0,c1_xor),np.bitwise_xor(c0_0,c1_1_xor)),axis=1)#解密一轮在异或
#         creal_1=np.concatenate((creal_1,np.bitwise_xor(c0_1,c1),np.bitwise_xor(c0_1,c1_1)),axis=1)
        # creal_0=np.concatenate((creal_0,c1,c1_1),axis=1)#解密一轮直接做
    #生成负样本
    x1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
    p1 = np.zeros((n, 64), dtype=np.uint8)
    x1_1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
    p1_1 = np.zeros((n, 64), dtype=np.uint8)
    for i in range(64):
        off = 63 - i
        p1[:, i] = (x1 >> off) & 1
        p1_1[:, i] = (x1_1 >> off) & 1
    c2=encrypt(p1, subkeys)
    c2_1=encrypt(p1_1, subkeys)

    c2_1_xor=np.bitwise_xor(c2,c2_1)

    c2_2_2,c2_2=decrypt_1(np.bitwise_xor(c2,c2))
    c2_1_xor_2,c2_1_xor_tran=decrypt_1(c2_1_xor)
    crand=np.concatenate((c2_2,c2_1_xor_tran),axis=1)
    crand_1=np.concatenate((c2_2_2,c2_1_xor_2),axis=1)
#     crand=np.concatenate((c2_2,np.bitwise_xor(c2_2,c2_1_xor)),axis=1)  #异或值
#     crand_1=np.concatenate((c2_1,np.bitwise_xor(c2,c2_1)),axis=1)  #异或值
    # crand_0=np.concatenate((c2,c2_1),axis=1)
    for j in range(7):
        x1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
        p1 = np.zeros((n, 64), dtype=np.uint8)
        x1_1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
        p1_1 = np.zeros((n, 64), dtype=np.uint8)
        for i in range(64):
            off = 63 - i
            p1[:, i] = (x1 >> off) & 1
            p1_1[:, i] = (x1_1 >> off) & 1
        c3=encrypt(p1, subkeys)
        c3_1=encrypt(p1_1, subkeys)

        c3_xor=np.bitwise_xor(c2,c3)
        c3_1_xor=np.bitwise_xor(c2,c3_1)

        c3_xor_tran_2,c3_xor_tran=decrypt_1(c3_xor)
        c3_1_xor_tran_2,c3_1_xor_tran=decrypt_1(c3_1_xor)

        crand=np.concatenate((crand,c3_xor_tran,c3_1_xor_tran),axis=1)
        crand_1=np.concatenate((crand_1,c3_xor_tran_2,c3_1_xor_tran_2),axis=1)
#         crand=np.concatenate((crand,np.bitwise_xor(c2_2,c3_xor),np.bitwise_xor(c2_2,c3_1_xor)),axis=1)
#         crand_1=np.concatenate((crand_1,np.bitwise_xor(c2_1,c3),np.bitwise_xor(c2_1,c3_1)),axis=1)
        # crand_0=np.concatenate((crand_0,c3,c3_1),axis=1)
    #生成负样本2


#     X = np.concatenate((creal_0,crand_0))
    X = np.concatenate((np.concatenate((creal,creal_1),axis=1),np.concatenate((crand,crand_1),axis=1)))
    Yreal  =  np. ones(n)
    Yrand  =  np. zeros(n)
    Y  =  np. concatenate((Yreal, Yrand))
    return X,Y
# verify(n=10, nr=31)
def make_train_data_mutil(nums,rounds):
    master_keys = np.frombuffer(urandom(nums * 80), dtype=np.uint8).reshape(-1, 80) & 1
    X,Y=make_train_data(master_keys,nums,rounds)
#     X_1,Y_1=make_train_data(master_keys,nums,rounds)
#     X_2,Y_2,crd_2=make_train_data(master_keys,nums,rounds)
#     X_3,Y_3,crd_3=make_train_data(master_keys,nums,rounds)
#     X_test=np.concatenate((X,X_1),axis=1)
    return X,Y

In [ ]:
def cyclic_lr(num_epochs, high_lr, low_lr):
    res = lambda i: low_lr + ((num_epochs - 1) - i % num_epochs) / (num_epochs - 1) * (high_lr - low_lr)
    return (res)


def make_checkpoint(datei):
    res = ModelCheckpoint(datei, monitor='val_loss', save_best_only=True)
    return (res)
def make_resnet(num_words=16, multiset=16, num_filters=1024, num_outputs=1, d1=2048, d2=2048, word_size=8, ks=3, depth=5,
                reg_param=0.0001, final_activation='sigmoid'):
    # Input and preprocessing layers
    inp = Input(shape=(num_words * 8 * multiset,))
    rs = Reshape((num_words * multiset, 8))(inp)
    perm = Permute((2, 1))(rs)
    # add a single residual layer that will expand the data to num_filters channels
    # this is a bit-sliced layer
    conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv0 = BatchNormalization()(conv0)
    conv0 = Activation('relu')(conv0)
    # add residual blocks
    shortcut = conv0
    for i in range(depth):
        conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Dropout(0.2)(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])
    # add prediction head
    flat1 = Flatten()(shortcut)
    dense1 = Dense(d1, kernel_regularizer=l2(reg_param))(flat1)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)
    dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)
    dense2 = Dropout(0.2)(dense2)
    out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)
    return (model)


def train_present_distinguisher(num_epochs, num_rounds, depth):
    # create the network
    try:
      tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    except ValueError:
      tpu = None

    # TPUStrategy for distributed training
    if tpu:
      tf.config.experimental_connect_to_cluster(tpu)
      tf.tpu.experimental.initialize_tpu_system(tpu)
      strategy = tf.distribute.experimental.TPUStrategy(tpu)
      print("train on tpu")
    else: # default strategy that works on CPU and single GPU
      strategy = tf.distribute.get_strategy()
    with strategy.scope():
        net = make_resnet(depth=depth, reg_param=0.00001)
        net.compile(optimizer='adam', loss='mse', metrics=['acc'])
    # generate training and validation data and test data
    print(1)
    X,Y=make_train_data_mutil(2**20,num_rounds)
    print(2)
    X_eval,Y_eval=make_train_data_mutil(2**16,num_rounds)
    print(3)
    # X_test, Y_test = make_train_data(2 ** 8, num_rounds)
    # create learnrate schedule
    lr = LearningRateScheduler(cyclic_lr(10, 0.002, 0.0001))
    # train and evaluate
    time_start = time.time()
    h  =  net.fit(X, Y, epochs=num_epochs, batch_size=bs, validation_data=(X_eval, Y_eval), callbacks=[lr])
    # loss, accuracy  =  net. evaluate(X_test, Y_test)
    time_end = time.time()
    total_time = time_end - time_start
    
    print("\nWhen training for a", num_rounds, "round PRESENT ", num_epochs, "epochs:")
    print("\nBest validation accuracy: ", np.max(h.history['val_acc']))
    # print('\nTest loss:', loss)
    # print('\nTest accuracy:', accuracy)
    # f = open(save_path + "result_for_lyu_train_PRESENT.txt", "a")
    print('\nTotal training time is: %.2f seconds.' % total_time)
    return (net, h)

In [ ]:
model, history = train_present_distinguisher(50, num_rounds=8, depth=10)

In [ ]:
X_test,Y_test=make_train_data_mutil(2**16,8)
loss, accuracy  =  model. evaluate(X_test, Y_test)
print('\nTest loss:', loss)
print('\nTest accuracy:', accuracy)